I'm going to rebuild my homelab, except I'm going to do it in the cloud.

I don't want to manage anything myself (yet). For now, I just want to deploy my private apps and services to my very own cloud LAN.

Gameplan:
1. [Deploy an app to GKE](https://cloud.google.com/kubernetes-engine/docs/deploy-app-cluster) https://tailscale.com/kb/1185/kubernetes#use-the-kubernetes-operator
2. [Access my app via Tailscale](https://tailscale.com/kb/1185/kubernetes#use-the-kubernetes-operator)
    - This is the "cloud LAN" part of my plan. I was inspired after reading David Crawshaw's [Remebering the LAN](https://crawshaw.io/blog/remembering-the-lan) post. I reflected on how magical my LAN was, and how I wanted to experience that magic without the pain of managing the machines on my LAN. In other words, I want to let someone else manage my infrastructure, but I don't want to lose the magic of the cloud LAN.
    - I feel like that rules out most PaaS offerings. They all seem dedicated on deploying production apps to the public. But I don't want that! I want to deploy my self hosted apps for no one except myself (and maybe my friends, if I am finally able to prevent my services from going down).

Prerequisites

- [`gcloud` CLI](https://cloud.google.com/sdk/gcloud) with [`gke-gcloud-auth-plugin`](https://cloud.google.com/kubernetes-engine/docs/how-to/cluster-access-for-kubectl#install_plugin)
- A GCP billing account

In [4]:
%%sh
# don't forget --billing-account=xxxxxx-xxxxxx-xxxxxx
gcloud projects create griffinht-cloudlab

Create in progress for [https://cloudresourcemanager.googleapis.com/v1/projects/griffinht-cloudlab].
Waiting for [operations/cp.7107272660974735241] to finish...
..done.
Enabling service [cloudapis.googleapis.com] on project [griffinht-cloudlab]...
Operation "operations/acat.p2-549404533601-f5b7134d-13b6-41b2-99c5-b3334cc6669c" finished successfully.


In [21]:
%env CLOUDSDK_CORE_PROJECT=griffinht-cloudlab
%env CLOUDSDK_COMPUTE_ZONE=us-east1

env: CLOUDSDK_CORE_PROJECT=griffinht-cloudlab
env: CLOUDSDK_COMPUTE_ZONE=us-east1


In [8]:
%%sh
gcloud container clusters create-auto my-cluster

ERROR: (gcloud.container.clusters.create-auto) ResponseError: code=403, message=Kubernetes Engine API has not been used in project griffinht-cloudlab before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/container.googleapis.com/overview?project=griffinht-cloudlab then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.


CalledProcessError: Command 'b'gcloud container clusters create-auto my-cluster \\\n    --location=us-east1\n'' returned non-zero exit status 1.

In [12]:
%%sh
gcloud services enable container.googleapis.com

Operation "operations/acf.p2-549404533601-dd498467-be11-4e22-a8b3-7cc461110abb" finished successfully.


In [16]:
%%sh
gcloud container clusters create-auto my-cluster

Creating cluster my-cluster in us-east1...
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

NAME        LOCATION  MASTER_VERSION      MASTER_IP     MACHINE_TYPE  NODE_VERSION        NUM_NODES  STATUS
my-cluster  us-east1  1.29.4-gke.1043002  35.227.21.72  e2-small      1.29.4-gke.1043002  3          RUNNING


It's up! Let's explore what we have created using the standard Kubernetes tools I explored in my zero to kubernetes lab.

Note that I had to go install [`gke-gcloud-auth-plugin`](https://cloud.google.com/kubernetes-engine/docs/how-to/cluster-access-for-kubectl#install_plugin) at this point.

In [24]:
%%sh
gcloud container clusters get-credentials my-cluster

Fetching cluster endpoint and auth data.
kubeconfig entry generated for my-cluster.


In [25]:
%%sh
kubectl cluster-info 

Kubernetes control plane is running at https://35.227.21.72
GLBCDefaultBackend is running at https://35.227.21.72/api/v1/namespaces/kube-system/services/default-http-backend:http/proxy
KubeDNS is running at https://35.227.21.72/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://35.227.21.72/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


Everything looks good.

Following my example from my todo zero to Kubernetes lab, I will be deploying `searxng`, a privacy focused metasearch engine.

I already know how I can test my `pod.yaml` definition locally, so I'm ready to deploy it to my cluster. 

In [26]:
%%sh
cat > pod.yaml << EOF
apiVersion: v1
kind: Pod
metadata:
    name: searxng
spec:
    containers:
      - name: searxng
        image: searxng/searxng
        ports:
        - containerPort: 8080
EOF

kubectl apply -f pod.yaml

pod/searxng created


This took a while to start up, but I was able to watch the progress from `k9s`.

![a picture of my pod running in k9s](k9s.png)

In [34]:
%%sh
kubectl get pods

NAME      READY   STATUS    RESTARTS   AGE
searxng   1/1     Running   0          6m26s


In [35]:
%%sh
kubectl exec searxng -- wget icanhazip.com

Connecting to icanhazip.com (104.16.185.241:80)
saving to 'index.html'
index.html           100% |********************************|    15  0:00:00 ETA
'index.html' saved


Looks like the pod has internet access. Now I'm going to access the pod from my Tailscale network (what I am calling my "Cloud LAN").

I'm going to pick up from my Tailscale Kubernetes lab

I want to be able to access `searxng` from `searxng.cloudlab.griffinht.com`



TAILSCALE IS NOT NEEDED I COULD HAVE JUST USED PLAIN OLD WIREGUARD!!!

https://kubernetes-sigs.github.io/external-dns/v0.14.2/tutorials/cloudflare/#using-helm

# Multi Environment Automation

Now that I have my environment up and running, I would like to 